## BART Parameter Spec
	
24-layer, 1024-hidden, 16-heads, 406M parameters

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

In [43]:
from transformers import BartForConditionalGeneration, BartTokenizer

# BART 모델과 토크나이저 로드
model_name = 'facebook/bart-large'
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

In [76]:

# CoT 프롬프트 정의
# CoT와 Few-shot 예시를 포함한 프롬프트 정의
prompt = f"""
What is the higher-level concept of the term 'Donald Trump'?
A: Person
What is the higher-level concept of the term 'Canada'?
A: Nation
What is the higher-level concept of the term 'Apartment'?
A: {tokenizer.mask_token}
"""

# 프롬프트를 토큰화하고 입력 시퀀스 생성
inputs = tokenizer(prompt, return_tensors='pt')

inputs


{'input_ids': tensor([[    0, 50118,  2264,    16,     5,   723,    12,  4483,  4286,     9,
             5,  1385,   128, 19195,   140, 35661, 50118,   250,    35, 18404,
         50118,  2264,    16,     5,   723,    12,  4483,  4286,     9,     5,
          1385,   128, 14031, 35661, 50118,   250,    35,  5857, 50118,  2264,
            16,     5,   723,    12,  4483,  4286,     9,     5,  1385,   128,
         36675, 27699, 35661, 50118,   250,    35, 50264, 50118,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [77]:
# 모델을 사용하여 추론 수행
outputs = model.generate(**inputs,num_beams=1, early_stopping=True)

# 생성된 응답을 디코딩
response = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(response)

/home/sjlee/miniconda3/envs/sj_virtual/lib/python3.8/site-packages/transformers/generation/configuration_utils.py:588: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
/home/sjlee/miniconda3/envs/sj_virtual/lib/python3.8/site-packages/transformers/generation/utils.py:1249: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


What is the higher-level concept of the term 'Donald Trump'? fixme


In [95]:
from nltk import ne_chunk,word_tokenize,pos_tag

In [96]:
word_tokenize("Apple is good company")

['Apple', 'is', 'good', 'company']

## WordNet을 이용한 개념의 상위개념 찾기

In [1]:
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /home/sjlee/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/sjlee/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [1]:
from nltk.corpus import wordnet as wn

In [38]:
word = "dinosour"
synsets = wn.synsets(word)
synsets

[Synset('speed.n.01'),
 Synset('speed.n.02'),
 Synset('speed.n.03'),
 Synset('focal_ratio.n.01'),
 Synset('amphetamine.n.01'),
 Synset('rush.v.01'),
 Synset('accelerate.v.01'),
 Synset('travel_rapidly.v.01'),
 Synset('speed.v.04'),
 Synset('accelerate.v.02')]

In [39]:
synsets[0].definition()

'distance travelled per unit time'

In [40]:
hypernyms = synsets[0].hypernyms()
hypernyms

[Synset('rate.n.01')]

In [41]:
hypernyms[0].definition()

'a magnitude or frequency relative to a time unit'

In [35]:
word = "San_diego"
synsets = wn.synsets(word)
synsets

[Synset('san_diego.n.01')]

In [36]:
synsets[0].definition()

'a picturesque city of southern California on San Diego Bay near the Mexican border; site of an important naval base'

In [37]:
hypernyms = synsets[0].hypernyms()
hypernyms

[]

In [ ]:
hypernyms[0].definition()

'a school for students intermediate between elementary school and college; usually grades 9 to 12'

In [20]:
for synset in synsets:
    print(f"{synset.name()} : {synset.definition()}")
    
    # 상위개념 찾기
    hypernyms = synset.hypernyms()
    print("  상위개념:")
    for hypernym in hypernyms:
        print(f"  {hypernym.name()} : {hypernym.definition()}")

paris.n.01 : the capital and largest city of France; and international center of culture and commerce
  상위개념:
paris.n.02 : sometimes placed in subfamily Trilliaceae
  상위개념:
  plant_genus.n.01 : a genus of plants
paris.n.03 : (Greek mythology) the prince of Troy who abducted Helen from her husband Menelaus and provoked the Trojan War
  상위개념:
paris.n.04 : a town in northeastern Texas
  상위개념:
